In [ ]:
using PowerModels
using Ipopt
using ForwardDiff
using OrdinaryDiffEq
using NLsolve

display(HTML("<style>.container { width:100% !important; }</style>"))

data = PowerModels.parse_file("C:/Users/Ewa/Downloads/Twogen2bus.m")
pf_result = run_ac_pf(data ,Ipopt.Optimizer)
#PowerModels.print_summary(pf_result["solution"])

# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg =pf_result["solution"]["gen"]["1"]["qg"]

# branch results
#p_ref =pf_result["solution"]["branch"]["1"]["pf"]
#q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

In [ ]:
PowerModels.print_summary(pf_result["solution"])

In [ ]:
p_ref = 0.3
q_ref = 0.03

In [ ]:
# Define the parameters
lf =0.08
rf = 0.03
cf = 0.074

rt =0.00005
lt = 0.15

Rp = 0.02
Rq =0.001
ωz = 2*pi*5.0

kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 0.0 ## should it be zero??

kp_pll =0.4
ki_pll =4.69

ω_ref = 1.0
v_ref = V_tR1
#ωn = 2*pi*50.0
ωn = 1.0
ωb = 2*pi*50.0



    function system!(F,x)
    
    pm = x[1]*x[5] + x[2]*x[6]
    qm = -x[1]*x[6] + x[2]*x[5]
    ϵ_dot = x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])
    ω_pll = ωn + kp_pll*ϵ_dot + ki_pll*x[9]
    ω_apc = ω_pll + Rp*(p_ref - x[10])
    # ω_apc = 1.0
    v_hat = v_ref + Rq*(q_ref - x[11])
    # v_hat = 1.065
    v_iref_d = v_hat - rv*x[5] + ω_apc*lv*x[6]
    v_iref_q = - rv*x[6] - ω_apc*lv*x[5]
    i_hat_d = kvp*(v_iref_d -x[1]) + kvi*x[12] - ω_apc*cf*x[2] + kif*x[5]
    i_hat_q = kvp*(v_iref_d -x[2]) + kvi*x[13] + ω_apc*cf*x[1] + kif*x[6]
    v_md = kip*(i_hat_d -x[3]) + kii*x[13] - ω_apc*lf*x[4] + kvf*x[1]
    v_mq = kip*(i_hat_q -x[4]) + kii*x[14] + ω_apc*lf*x[3] + kvf*x[2]
    v_gd = V_tR1*cos(x[7])+V_tI1*sin(x[7])
    v_gq =-V_tR1*sin(x[7])+V_tI1*cos(x[7])
    ωg = ω_apc
    
    
###Grid following equations
    #𝜕ed/𝜕t
    F[1]= (ωb / cf)*(x[3] - x[5]) + ωg*ωb*x[2]
    #𝜕eq/𝜕t
    F[2]= (ωb / cf)*(x[4] - x[6]) - ωg*ωb*x[1]
    #𝜕isd/𝜕t
    F[3]= (ωb / lf)*(v_md - x[1]) - (rf*ωb/lf)*x[3] + ωb*ωg*x[4]
    #𝜕isq/𝜕t
    F[4]= (ωb / lf)*(v_mq - x[2]) - (rf*ωb/lf)*x[4] - ωb*ωg*x[3]
    #𝜕igd/𝜕t
    F[5]= (ωb / lt)*(x[1] - v_gd) - (rt*ωb/lt)*x[5] + ωb*ωg*x[6]
    #𝜕igq/𝜕t
    F[6]= (ωb / lt)*(x[2] - v_gq) - (rt*ωb/lt)*x[6] - ωb*ωg*x[5] 
    #𝜕δc/𝜕t
    F[7] = ωb*(ω_apc - ωg)
    #𝜕δpll/𝜕t
    F[8] =ωb*(ω_pll- ωg)
    #𝜕ϵ_pll/𝜕t
    F[9] = ϵ_dot
    #𝜕pm/𝜕t
     F[10] = ωz*(pm- x[10])
    #𝜕qm/𝜕t
    F[11] = ωz*(qm - x[11])
    #𝜕ξ_d/𝜕t
    F[12] = v_iref_d - x[1]
    #𝜕ξ_q/𝜕t
     F[13] = v_iref_q - x[2]
    #𝜕γ_d/𝜕t
     F[14] = i_hat_d - x[3]
    #𝜕γ_q/𝜕t
    F[15] =  i_hat_q - x[4]
        
    F[16]= - pg+v_gd*x[5] + v_gq*x[6] 
end
# Initial Conditions
     initial =  [  V_tR1, #egd
            V_tI1, #egq
            p_ref/V_tR1, #isd  
            q_ref/V_tI1, #isq
            p_ref/V_tR1, #igd
            q_ref/V_tI1,#igq
            0.2, #δc
            0.0, #δpll
            0.0, #ϵ_pll
            p_ref, #pm
            q_ref, #qm
            0.0, #ξ_d
            0.0, #ξ_q
            0.0, #γ_d
            0.0, #γ_d
            0.0,
    ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                     xtol = :1e-9,
                     ftol = :1e-9,
                     method = :trust_region,
                    ) #Solve using initial guess x0
                      print(sys_solve)

In [ ]:
x = sys_solve.zero  

In [ ]:
v_gd = V_tR1*cos(x[7])+V_tI1*sin(x[7])

In [ ]:
v_gq=-V_tR1*sin(x[7])+V_tI1*cos(x[7])

In [ ]:
 ω_pll = ωn + kp_pll*(x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])) + ki_pll*x[9]

In [ ]:
ϵ_dot = x[1]*sin(x[7] - x[8]) + x[2]*cos(x[8] -x[7])

In [ ]:
 ω_apc = ω_pll + Rp*(p_ref - x[10])

In [ ]:
v_hat = v_ref + Rq*(q_ref - x[11])